In [9]:
import zipfile
import os
import sys
import xml.etree.ElementTree as ET
import subprocess
import copy
from unit_maker import UnitMaker
import numpy as np
os.listdir()

['.ipynb_checkpoints',
 'Add unit.ipynb',
 'blank.xml',
 'Flowsheet.xml',
 'old',
 'run_flowsheet.fsd',
 'sleek_version.ipynb',
 'TAC test',
 'test.fsd',
 'Tester',
 'unit_maker.py',
 '__pycache__']

## Configure initial file

In [23]:
etree=ET.parse('blank.xml')
root = etree.getroot()
etree.write('Flowsheet.xml')
with zipfile.ZipFile('run_flowsheet.fsd', 'w') as myzip:
    myzip.write('Flowsheet.xml')

## For extracting stuff to fiddle

In [2]:
"""
# Sometimes need this to quickly unzip to raw flosheet
zip_ref = zipfile.ZipFile("run_flowsheet.fsd", 'r')
zip_ref.extractall()
zip_ref.close()
"""
None

# Create a function for adding a new distillation column to an exisitng stream on a flowsheet

In [3]:
def add_column(stream_number, filename="run_flowsheet.fsd", **kwargs):
    zip_ref = zipfile.ZipFile(filename, 'r')
    zip_ref.extractall()
    zip_ref.close()
    # get etree
    etree=ET.parse("Flowsheet.xml")
    root = etree.getroot()
    
    # get inlet stream and relevant info
    stream = root.find(f"""stream[@name="{stream_number}"]""")
    end_coordinates = (int(stream.find("end").attrib["x"]), 
                       int(stream.find("end").attrib["y"]))
    Temperature = float(stream.find("temperature").text)
    Pressure = float(stream.find("pressure").text)
    total_flow = float(stream.find("molarFlow").text)/1000 # make in kmol/hr
    component_flow = [float(x)*total_flow for x in 
                    stream.find('phase[@name="Overall"]/moleFraction').text.split(";")]
    vapour_fraction = float(stream.find('phase[@name="Vapor"]/molarPhaseFraction').text)
    inlet_stream_info = {"number":stream_number,
                     "Temperature": Temperature,
                     "Pressure": Pressure,
                     "vapour_fraction": vapour_fraction,
                     "component_flow" : component_flow,
                     "end_coordinates": end_coordinates,
                    }
    column = UnitMaker(inlet_stream_info, **kwargs) # this is a class that adds a new DC column and relevant streams
    ChemSep_element = ET.fromstring(column.unit_string)
    tops_stream_element = ET.fromstring(column.top_stream_string)
    bottoms_stream_element = ET.fromstring(column.bottom_stream_string)
    
    root.append(tops_stream_element)
    root.append(bottoms_stream_element)
    root.append(ChemSep_element)
    
    etree.write('Flowsheet.xml')
    with zipfile.ZipFile('run_flowsheet.fsd', 'w') as myzip:
        myzip.write('Flowsheet.xml')
    return subprocess.run("Cofestand64 run_flowsheet.fsd", shell=True)
    

In [4]:
add_column(1, column_number=1)

CompletedProcess(args='Cofestand64 run_flowsheet.fsd', returncode=0)

In [5]:
add_column(211, column_number=2, n_stages = 27)

CompletedProcess(args='Cofestand64 run_flowsheet.fsd', returncode=0)

In [6]:
add_column(212, column_number=3, pressure_increase = 101325)

CompletedProcess(args='Cofestand64 run_flowsheet.fsd', returncode=0)

In [7]:
add_column(321, column_number=4)

CompletedProcess(args='Cofestand64 run_flowsheet.fsd', returncode=0)

# Inspect outputs
Need to find a way to grab data about the columns (most notably Total Annual Cost)

## Make Changes to final file

In [8]:
zip_ref = zipfile.ZipFile('run_flowsheet.fsd', 'r')
zip_ref.extractall()
zip_ref.close()

## Timed Function

All of the time taken by COFESTand - moving things around is very quick

In [24]:
import time
def add_column(stream_number, filename="run_flowsheet.fsd", **kwargs):
    start = time.time()
    zip_ref = zipfile.ZipFile(filename, 'r')
    zip_ref.extractall()
    zip_ref.close()
    # get etree
    etree=ET.parse("Flowsheet.xml")
    root = etree.getroot()
    
    # get inlet stream and relevant info
    stream = root.find(f"""stream[@name="{stream_number}"]""")
    end_coordinates = (int(stream.find("end").attrib["x"]), 
                       int(stream.find("end").attrib["y"]))
    Temperature = float(stream.find("temperature").text)
    Pressure = float(stream.find("pressure").text)
    total_flow = float(stream.find("molarFlow").text)/1000 # make in kmol/hr
    component_flow = [float(x)*total_flow for x in 
                    stream.find('phase[@name="Overall"]/moleFraction').text.split(";")]
    vapour_fraction = float(stream.find('phase[@name="Vapor"]/molarPhaseFraction').text)
    inlet_stream_info = {"number":stream_number,
                     "Temperature": Temperature,
                     "Pressure": Pressure,
                     "vapour_fraction": vapour_fraction,
                     "component_flow" : component_flow,
                     "end_coordinates": end_coordinates,
                    }
    column = UnitMaker(inlet_stream_info, **kwargs)
    ChemSep_element = ET.fromstring(column.unit_string)
    tops_stream_element = ET.fromstring(column.top_stream_string)
    bottoms_stream_element = ET.fromstring(column.bottom_stream_string)
    
    root.append(tops_stream_element)
    root.append(bottoms_stream_element)
    root.append(ChemSep_element)
    
    etree.write('Flowsheet.xml')
    middle = time.time()
    with zipfile.ZipFile('run_flowsheet.fsd', 'w') as myzip:
        myzip.write('Flowsheet.xml')
    subprocess.run("Cofestand64 run_flowsheet.fsd", shell=True)
    end = time.time()
    return start, middle, end

In [25]:
start, middle, end = add_column(1, column_number=1)

In [26]:
middle - start

0.015630006790161133

In [27]:
end - middle

20.872440576553345

In [18]:
start, middle, end = add_column(211, column_number=2, n_stages = 27)

In [19]:
middle - start

0.03047466278076172

In [20]:
end - middle

21.585485458374023